# Rockley Bio-Sensor Selection Codesign Project

## Mount Google Drive and configure PYTHONPATH variable

**Before running:** Find the `optimized_bio_sensor` folder in Drive and use the `Add shortcut to Drive` option to add it to `MyDrive`

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
# os.environ['PYTHONPATH'] += ':/content/gdrive/MyDrive/22FL/CS101-TA/optimized_bio_sensor/RockleyCodesign'
os.environ['PYTHONPATH'] += ':/content/gdrive/MyDrive/optimized_bio_sensor/RockleyCodesign'
os.environ['PYTHONPATH'] += ':/content/gdrive/MyDrive/optimized_bio_sensor/rockley'
# %cd /content/gdrive/MyDrive/22FL/CS101-TA/optimized_bio_sensor/RockleyCodesign
%cd /content/gdrive/MyDrive/optimized_bio_sensor/RockleyCodesign

Mounted at /content/gdrive
/content/gdrive/.shortcut-targets-by-id/1EaHvULJYewcfEpQ7wlYgmG1eRbVcahUv/optimized_bio_sensor/RockleyCodesign


## Install necessary packages

In [2]:
!pip install pytorch
!pip install torchvision
!pip install pytorch-lightning
!pip install wandb
!pip install fvcore

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
    Running setup.py install for pytorch ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-cfr1_5yt/pytorch_9ae305b410754774be1c608c2f4f8c77/setup.py'"'"'; __file__='"'"'/tmp/pip-install-cfr1_5yt/pytorch_9ae305b410754774be1c608c2f4f8c77/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-sfrm8k0h/install-record.txt --single-version-externally-managed --compile --install-headers /usr/local/include/python3.8/pytorch Check the logs f

## Login to Weights and Biases

Using Weights and biases is optional but highly recommended. It is an awesome platform for benchmarking, visualizing, and presenting results. To login, you need to create an account on https://wandb.ai/ and copy your API key. If you choose not to use it, comment line 32 in `codesign/engines/train_val_test.py`

In [ ]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## Train a model

To train a model is easy. Simply run `!python3 scripts/main.py -c CONFIG_FILE`, where `CONFIG_FILE` is the experiment configuration file in [YAML](https://en.wikipedia.org/wiki/YAML) format. The example below is a classic FashionMNIST classification experiment. Take a look at how to configurate the experiment in `configs/fashion_mnist_classification_demo.yaml`!





Current default stopping criterion: the training will be stopped if either `val_test_loss` in config does not improve for 10 epochs (patience=10) or the epoch reaches `trainer.max_epochs` in config, whichever happens earlier. You can the patience on line 22 in `codesign/engines/train_val_test.py`. 



Codesign models use LOUPESampler as their sampler in their config file (or SetLOUPESampler for CNN but this is slow and results aren't particularly good)

See `configs/LOUPEModel.yaml` for more details on particular specifications

While training, PyTorch lightning will save the best model its seen so far in terms of val loss

After a bit, you can use code from TestNNModels notebook to visualize the probability map from that checkpoint

If the probability map stabilizes to a binary map (we saw that it did), you can then pull those indices in that stable binary map and use Subsampler as a sampler instead. This will only pass through those indices which allows you to downsize input size to regressor while keeping the same effect (helps with convergence + speed). See `configs/PreindexedLOUPEModel.yaml` for an example/directions

At any point, you can stop training, tweak learning/batch rate/etc. to help speed up training, and re-run from that checkpoint by setting init conditions in those config files

In [ ]:
!python3 scripts/main.py -c configs/LOUPEModel.yaml

wandb: Currently logged in as: donovan-ye. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.13.5
wandb: Run data is saved locally in results/rockley_nonlinear_regression/wandb/run-20221122_040953-2jvu6e76
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run rockley_nonlinear_regression
wandb: ⭐️ View project at https://wandb.ai/donovan-ye/rockley_codesign
wandb: 🚀 View run at https://wandb.ai/donovan-ye/rockley_codesign/runs/2jvu6e76
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:604: UserWarning: Checkpoint directory /content/gdrive/.shortcut-targets-by-id/1EaHvULJYewcfEpQ7wlYgmG1eRbVcahUv/optimized_bio_sensor/RockleyCodesign/results/rockley_nonlinear_regression exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and

## Test a model

After you train a model, your model (best performance with `val_test_loss`) will be saved in a folder with the same name as the `exp_name` in your config file. To test the model, run `!python3 scripts/main.py -c RUN_CONFIG_FILE -d DATA_CONFIG_FILE`, where `RUN_CONFIG_FILE` is the config file of the run and `DATA_CONFIG_FILE` is the config file of the data you want to test on. Usually, the test data is from the same dataset as the training/validation data, but in this way we allow mix-and-match training and testing by separating the two config files.






***Remember, test a model only after parameter tuning and model selection to avoid biases!***

In [3]:
!python3 scripts/test.py -c results/PreindexedLOUPEModel/config.yaml -d configs/default_testset_configs/PreindexedLOUPEModel.yaml

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
-------> test set: 40000 batches of size 1 (40000 samples in total) <-------
Testing DataLoader 0: 100% 40000/40000 [02:30<00:00, 264.92it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_mse_loss          23.66487693786621
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


## Beyond (it's on you!)

This example gives you an idea about how to start this codesign project. Now the stage is yours. Overall, you need to build this repo in the following ways:
  - Write data class for loading the Rockley data in `codesign.data`
  - Design customizd sampler for measurement selection in `codesign.samplers`
  - Implement classification/regression algorithms in `codesign.predictors`
  - Add new loss functions in `codesign.losses`
  - Write configuration files according to your needs in `configs`

Besides the essential to-dos above, you should also consider:
  - Adding visualizations in validation and test procedures in `codesign.engines.callbacks`